# 다중-인풋과 다중-아웃풋 모델

다중 인풋과 아웃풋을 다루는 모델은 함수형 API를 사용하기 특히 적합한 사례입니다. 함수형 API를 사용해서 복잡하게 얽힌 많은 수의 데이터 줄기를 간편하게 관리할 수 있습니다.

다음의 모델을 고려해 봅시다. 얼마나 많은 트위터에서 특정 뉴스 헤드라인이 얼마나 낳은 리트윗과 좋아요를 받을지 예측하려고 합니다. 모델에 대한 주요 인풋은 단어 시퀀스로 표현된 헤드라인 자체이지만, 문제를 조금 더 흥미롭게 풀기 위해, 보조 인풋으로 헤드라인이 올려진 시간 등과 같은 추가 데이터를 받도록 합시다. 두 개의 손실 함수를 통해 모델을 감독합니다. 주요 손실 함수를 모델의 초기 단계에 사용하는 것이 심화 모델에 있어 적절한 정규화 메커니즘입니다.

이 모델은 다음과 같이 구성됩니다:

![](https://s3.amazonaws.com/keras.io/img/multi-input-multi-output-graph.png)


In [1]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
import keras

Using TensorFlow backend.


In [2]:
# 헤드라인 인풋: 1에서 10000사이의 100개 정수로 이루어진 시퀀스를 전달받습니다
# "name"인수를 전달하여 레이어를 명명할 수 있음을 참고하십시오.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# 이 임베딩 레이어는 인풋 시퀀스를
# 밀집 512 차원 벡터의 시퀀스로 인코딩합니다
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

In [3]:
# 장단기 메모리는 벡터 시퀀스를 전체 시퀀스에 대한
# 정보를 포함하는 단일 벡터로 변형합니다
lstm_out = LSTM(32)(x)

In [4]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [5]:
auxilary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxilary_input])

In [6]:
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [7]:
model = Model(inputs=[main_input, auxilary_input], 
              outputs=[main_output, auxiliary_output])

In [8]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
             loss_weights=[1., 0.2])

In [9]:
model.fit([headline_data, additional_data], [labels, labels])

NameError: name 'headline_data' is not defined


```
model.compile(optimizer='rmsprop',
             loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})
```
또한 모델을 다음과 같이도 학습시킬 수 있습니다:
<pre><code>
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)
</code></pre>